<a href="https://colab.research.google.com/github/mrasw/Big-Data/blob/main/Assignment%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=4780a77f86024ab3c32230fecd1db284875673ff21f5eed2710224974ef747b5
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Check spark session
print(spark)

In [5]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!mkdir ~/.kaggle/
!touch ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [16]:
!mkdir ~/.kaggle/
!touch ~/.kaggle/kaggle.json

# PLEASE USE YOUR OWN KEY
# Download your own key according to this instruction https://github.com/Kaggle/kaggle-api#api-credentials

api_token = {"username":"mrpingu","key":"251e54438cab412c5edae438cdf984f3"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [19]:
# Download from https://www.kaggle.com/datasets/urbanbricks/wikipedia-promotional-articles

!kaggle datasets download -d urbanbricks/wikipedia-promotional-articles

 98% 198M/201M [00:10<00:00, 22.9MB/s]
100% 201M/201M [00:10<00:00, 20.6MB/s]


In [20]:
!unzip wikipedia-promotional-articles.zip

Archive:  wikipedia-promotional-articles.zip
  inflating: good.csv                
  inflating: promotional.csv         


In [21]:
!ls -la

total 783144
drwxr-xr-x 1 root root      4096 Mar 31 06:05 .
drwxr-xr-x 1 root root      4096 Mar 31 05:45 ..
drwxr-xr-x 4 root root      4096 Mar 29 16:45 .config
-rw-r--r-- 1 root root 475685227 Oct 27  2019 good.csv
-rw-r--r-- 1 root root 115360355 Oct 27  2019 promotional.csv
drwxr-xr-x 1 root root      4096 Mar 29 16:45 sample_data
-rw-r--r-- 1 root root 210863294 Mar 31 06:05 wikipedia-promotional-articles.zip


In [22]:
# Read CSV
df = spark.read.option("header", True).csv("/content/promotional.csv")
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- advert: string (nullable = true)
 |-- coi: string (nullable = true)
 |-- fanpov: string (nullable = true)
 |-- pr: string (nullable = true)
 |-- resume: string (nullable = true)
 |-- url: string (nullable = true)



In [23]:
# Add an ID for the dataset
from pyspark.sql.functions import monotonically_increasing_id

newsDF = df.withColumn("id", monotonically_increasing_id())
newsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+
|                text|advert|coi|fanpov| pr|resume|                 url| id|
+--------------------+------+---+------+---+------+--------------------+---+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|
|2N Telekomunikace...|     1|  0|     0|  0|     0|https://en.wikipe...|  7|
|A 3D printing mar...|     1|  0|     0|  0|     0|https://en.wikipe...|  8|
|3DR is an America...|     1|  1|     0|  0|     0|https://en.wikipe...|  9|

In [24]:
# Get the total rows
newsDF.count()

23837

In [25]:
# Prepare the tokenizer
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsDF = tokenizer.transform(newsDF)

wordsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|[1st, round, ente...|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|[2ergo, is, a, pr...|
|2N Telekomunikace...| 

In [26]:
# Vectorize the dataset
from pyspark.ml.feature import CountVectorizer

vocabSize=1000

# Train the CountVectorizer Model using our data
cvModel = CountVectorizer(inputCol="words", outputCol="features", vocabSize=vocabSize, minDF=10).fit(wordsDF)

# Transform our data into vector
vectorizedDF = cvModel.transform(wordsDF)
vectorizedDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|(1000,[0,1,2,3,4,...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|

In [27]:
from  pyspark.ml.feature import MinHashLSH

mh = MinHashLSH(inputCol="features", outputCol="hashValues", numHashTables=3)
LSHmodel = mh.fit(vectorizedDF)

LSHmodel.transform(vectorizedDF).show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|          hashValues|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|[[4.735301E7], [2...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|[[5942687.0], [3....|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|[[2609176.0], [1....|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|[[1341540.0], [1....|
|The Jerusalem Bie...| 

In [28]:
print(cvModel.vocabulary.index("united"))
print(cvModel.vocabulary.index("states"))

92
198


In [29]:
# Testing searching for "united" "states"

from pyspark.ml.linalg import Vectors


# Convert the input with 3 words into 1000 size vectors
# If the words exist in the index we will give value = 1.0, otherwise 0.0
# Final result: key = [0, 0, ... , 1.0, ..., 1.0, 1.0, ....]

key = Vectors.sparse(vocabSize, {cvModel.vocabulary.index("civil"): 1.0, cvModel.vocabulary.index("war"): 1.0})

In [30]:
# Define the number of neighbours
k = 40

# Search inside LSH model that we already trained
resultDF = LSHmodel.approxNearestNeighbors(vectorizedDF, key, k)
resultDF.show()

+----+------+---+------+---+------+---+---+-----+--------+----------+-------+
|text|advert|coi|fanpov| pr|resume|url| id|words|features|hashValues|distCol|
+----+------+---+------+---+------+---+---+-----+--------+----------+-------+
+----+------+---+------+---+------+---+---+-----+--------+----------+-------+



In [31]:
# Save the result into CSV
import pandas as pd

data = resultDF.toPandas()
data.to_csv("result.csv")